# Create Annual Calibrated Composite Images for Each RTS Polygon

## Set Up Environment

In [ ]:
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap
import os
import json
from pprint import pprint
import math
import statistics
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
import xarray as xr
import rioxarray as rxr
from datetime import datetime
from collections import Counter
import re
from google.cloud import storage

In [ ]:
# Set up access to abrupt_thaw
storage_client = storage.Client(project="AbruptThawMapping")
abrupt_thaw = storage_client.get_bucket('abrupt_thaw')

## Define Functions

In [ ]:
# apply image IDs to each image in the collection
def setID(image):
    img_id = image.id();
    img_prop = image.setMulti({'ID': img_id});
    img_prop = ee.Image(img_prop);
    return img_prop;


In [ ]:
# Mask the data to values greater than 0 (which is how nodata gets imported into GEE by default)
def mask_0(image):
    mask = image.gt(0)
    return image.updateMask(mask)

In [ ]:
# function to get UTM zone from WGS84 lat and lon
def utm_from_wgs84(lon, lat):
    #Special Cases for Norway and Svalbard
    if (lat > 55 and lat < 64 and lon > 2 and lon < 6):
        return 32
    elif (lat > 71 and lon >= 6 and lon < 9):
        return 31
    elif (lat > 71 and ((lon >= 9 and lon < 12) or (lon >= 18 and lon < 21))):
        return 33
    elif (lat > 71 and ((lon >= 21 and lon < 24) or (lon >= 30 and lon < 33))):
        return 35
    # Rest of the world
    elif (lon >= -180 and lon <= 180):
        return 32600 + (math.floor((lon + 180) / 6) % 60) + 1 # 32600 for northern hemisphere
    else:
        raise ValueError('Cannot figure out UTM zone from given Lat: {0}, Lon: {1}.'.format(lat, lon))

## Import Data and Prepare Visualization Parameters

In [ ]:
# Import Planet Data GCS
planet = ee.ImageCollection('projects/abruptthawmapping/assets/nitze_regions_imagery_calibrated')
planet = planet.map(setID)

In [ ]:
# View the imagery
vis_params_imagery = {
    'min': 0,'max': 1200,
    'bands': ['red', 'green', 'blue'],
    'gamma': 0.9
}

## Create Composites

In [ ]:
# create a composite image across all regions for just 2019 and for 2018 and 2019 combined
planet_2019 = planet.filter(ee.Filter.stringContains('ID', '2019'))

planet_composites_all = planet.median()
planet_composites_2019 = planet_2019.median()

## Map Composites

In [ ]:
# Prep Map
Map = geemap.Map()
Map.centerObject(planet_composites_2019)

In [ ]:
# Add composites to the map as one layer
Map.addLayer(ee.ImageCollection(planet_composites_2019),
             vis_params_imagery,
             '2019 Composites')
Map.addLayer(ee.ImageCollection(planet_composites_all),
             vis_params_imagery,
             '2018 +2019 Composites')

In [ ]:
Map

## Export Annual Composites

In [ ]:
# Import shapefile with AOI (multipolygon)
aoi = gpd.read_file("/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/nitze_regions/bboxes/nitze_bbox_water_removed.shp")
# convert from multipolygon to multiple polygons
aoi = aoi.explode(column = 'geometry', ignore_index = True)
# remove inner holes
aoi.geometry = aoi.geometry.exterior
# convert back to polygon
aoi.geometry = [shp.geometry.Polygon([shp.geometry.Point(x, y) for x, y in list(feature.coords)]) for feature in aoi.geometry]
aoi['region'] = list(range(0, 7))
# convert to json for planet data search
sites = json.loads(aoi.to_json()) # if multiple sites
aoi

In [ ]:
zones = pd.DataFrame(columns = ['region', 'utm_zone'])
for idx, region in enumerate(aoi.geometry):
    region_zones = []
    for x, y in zip(region.exterior.coords.xy[0], region.exterior.coords.xy[1]):
        region_zones.append(utm_from_wgs84(x, y))
        
    region_zones = round(statistics.median(region_zones))
    temp_df = pd.DataFrame({'region': [idx],
                            'utm_zone': [region_zones]})
    zones = pd.concat([zones, temp_df])
zones = zones.set_index('region')  
zones

In [ ]:
# Export Composites to Drive
for row in aoi.iterrows():
    pprint(row)
    region = row[1]['region']
    name = 'nitze_regions_' + str(region) + '_2019_composite'
    geometry = sites['features'][region]['geometry']['coordinates']
    crs = 'EPSG:' + str(zones.iloc[region].utm_zone)
    print(crs)
    task = ee.batch.Export.image.toCloudStorage(
        image = planet_composites_2019,
        description = name,
        bucket = 'abrupt_thaw',
        fileNamePrefix = 'planet_processing/data/nitze_regions/calibrated_composites/' + name,
        crs = crs,
        region = geometry,
        scale = 3,
        maxPixels = 1e13,
        fileFormat = 'GeoTIFF',
        formatOptions = {'cloudOptimized': True}
    )
    task.start()

In [ ]:
# Export Composites to Drive (composites include 2018 images)
for row in aoi.iterrows():
    pprint(row)
    region = row[1]['region']
    name = 'nitze_regions_' + str(region) + '_all_composite'
    geometry = sites['features'][region]['geometry']['coordinates']
    crs = 'EPSG:' + str(zones.iloc[region].utm_zone)
    print(crs)
    task = ee.batch.Export.image.toCloudStorage(
        image = planet_composites_all,
        description = name,
        bucket = 'abrupt_thaw',
        fileNamePrefix = 'planet_processing/data/nitze_regions/calibrated_composites/' + name,
        crs = crs,
        region = geometry,
        scale = 3,
        maxPixels = 1e13,
        fileFormat = 'GeoTIFF',
        formatOptions = {'cloudOptimized': True}
    )
    task.start()